In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt


#constants
layers = 3
total = 7291
ita = 0.0001
parameters = 256
m = int(parameters*0.5)

def randomW(lenX, lenY, m, layers):
    W = []
    W.append(np.random.uniform(low=-1, high=1, size= (m,lenX+1)))
    W.append(np.random.uniform(low=-1, high=1, size= (lenY,m+1)))
    return W

def fwd_propagation(X, W_NN, m, layers):
    S_NN = []
    X_NN = []
    s = []
    X_temp = []

    #insert 1 for X
    X = np.insert(X,0,1)
    X_NN.append(X)
    for i in range(0, layers-1):
        #s(l) = W(l)*X(l-1)
        s = np.dot(W_NN[i], X_NN[i])
        S_NN.append(s)

        X_temp = np.tanh(s)
        if(i != layers-2):
            X_temp = np.concatenate([[1], X_temp])
        X_NN.append(X_temp)
    return X_NN

def back_propagation(X_NN, W_NN, Y, m, layers):
    theta = []
    delta_NN = []

    #del(L) = 2*(X(L)-Y)*(1-X(L)^2)
    theta = np.array(1 - (np.square(X_NN[-1])))
    delta_NN.append((2 * (np.dot(np.subtract(X_NN[-1],Y),theta))))
    delta_NN = np.array(delta_NN)

    for i in range(layers-2,0,-1):
        #W(l+1).delta_NN(l+1)
        temp = np.dot(delta_NN[0], W_NN[i])
        #theta = 1-X(l)^2
        theta = np.subtract(1, np.square(X_NN[i]))
        
        delta =[]
        #delta_NN[i] = theta*W(l+1).delta_NN(l+1)
        for j in range(1,len(theta)):
            delta.append(theta[j]*temp[0][j])
        delta_NN = np.array([np.array(delta), delta_NN])
    return delta_NN

def classifyDigits():
    #Initialize parameters G_NN, E_in, W_NN

    #initialize G_NN to 0 and E_in to 0
    a = (m,parameters+1)
    b = (1,m+1)
    G_NN = []
    G_NN.append(np.zeros(a))
    G_NN.append(np.zeros(b))
    G_NN = np.array(G_NN)
    E_in = []
    E_in_last = 0
    traincount = 0
    testcount = 0
    testpass = 0
    testfail = 0
    #initialize W to random array
    W_NN = randomW(256, 1, m, layers)
    X_NN = []
    delta_NN = []

    #open training file
    with open("C:/Akshata/Courses/Machine_Learning/Homeworks/ML_HW4/Digits/Digits/ZipDigits.train", 'r') as fp:
        #read a line as string
        for line in fp:
            traincount+=1
            X = []
            Y = []
            #split and convert to array of floats
            data = [float(i) for i in line.split()]
            Y = data[:1]
            X = data[1:]
            X = np.array(X)
            #forward propagation
            X_NN = fwd_propagation(X, W_NN, m, layers)
            #back propagation
            delta_NN = back_propagation(X_NN, W_NN, Y, m, layers)
            
            #calculate gradient and update W based on the result
            for i in range(layers-1):
                #Converting to 2-D arrays from 1-D array for dot product
                if X_NN[i].ndim == 1:
                    X_NN[i] = X_NN[i][:,np.newaxis]
                if delta_NN[i].ndim == 1:
                    delta_NN[i] = delta_NN[i][:,np.newaxis]
                
                #Calculate gradient and update W as per gradient
                G_NN[i] = np.dot(delta_NN[i], np.transpose(X_NN[i]))

            W_NN -= ita*G_NN

            E_in_last = ((X_NN[-1][0] - Y[0])**2)
            E_in.append(E_in_last)

    #Open test files
    with open("C:/Akshata/Courses/Machine_Learning/Homeworks/ML_HW4/Digits/Digits/ZipDigits.test", 'r') as fp:
        for line in fp:
            testcount+=1
            X = []
            Y = []
            #split and convert to array of floats
            data = [float(i) for i in line.split()]
            Y = data[:1]
            X = data[1:]
            X = np.array(X)
            #forward propagation
            X_NN = fwd_propagation(X, W_NN, m, layers)
            if Y[0] == 1.00:
                if X_NN[-1][0] >= 0.9 and X_NN[-1][0] <= 1.1:
                    testpass+=1
                else:
                    testfail+=1
        print("total:",testcount, 'testpass:', testpass, 'testfail:',testfail)
    return E_in
    
E_in = classifyDigits()
t = [n for n in range(0,len(E_in))]
plt.plot(E_in, t, 'ro')
plt.xlabel(t)
plt.ylabel(E_in)
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Akshata/Courses/Machine_Learning/Homeworks/ML_HW4/Digits/Digits/ZipDigits.train'